In [1]:
import pandas as pd
from dateutil.parser import parse
import ephem
    
import spacy
import pytextrank
from keyword_spacy import KeywordExtractor


nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("textrank")
nlp.add_pipe("keyword_extractor", last=True, config={"top_n": 10, "min_ngram": 3, "max_ngram": 3, "strict": True, "top_n_sent": 3})


from spacy.language import Language
from spacy_language_detection import LanguageDetector

# import nltk
# nltk.download('vader_lexicon')
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
from rake_nltk import Rake
import nltk
nltk.download('stopwords')


def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)  # We use the seed 42
Language.factory("language_detector", func=get_lang_detector)

nlp.add_pipe('language_detector', last=True)

/home/daniel/odm/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
[nltk_data] Downloading package stopwords to /home/daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
sun = pd.read_csv('sunspots.csv')
def update_date(row):
    # print(row)
    text_date = str(row['day']) + "/" + str(row['month']) + "/" + str(row['year'])
    # print(text_date)
    
    parsed_date = parse(text_date)
    m = ephem.Moon(parsed_date)
    return {
        'formatted_date': parsed_date.strftime('%d/%m/%Y'),
        'moon_phase': m.moon_phase
    }
applied_df = sun.apply(lambda row: update_date(row), axis='columns', result_type='expand')
sun = pd.concat([sun, applied_df], axis='columns')

In [3]:
sample_size = "all"
# sample_size = 2222
odf = pd.read_csv('ln_odm_database_mvp.csv')

# df = odf.head(5000)
odf.dropna(subset=['date'], inplace=True)
odf.dropna(subset=['description'], inplace=True)
if sample_size == "all":
    df = odf
else:
    df = odf.sample(n=sample_size)    



In [4]:
def get_date_data(date):
    parsed_date = parse(date)
    m = ephem.Moon(parsed_date)
    ss = sun.loc[sun['formatted_date'] == parsed_date.strftime('%d/%m/%Y')]
    if ss['snvalue'].any():
        sss = ss['snvalue'].values[0]
    else:
        sss = ''
    
    return {
        'sunspots': sss,
        'formated_date': parsed_date.strftime('%d/%m/%Y'),
        'moon_phase': m.moon_phase
    }
applied_df = df.apply(lambda row: get_date_data(row.date), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')
df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,contact_type_raw,method_raw,sunspots,formated_date,moon_phase
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,NaN,NaN,11,31/03/2020,0.453401
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,NaN,NaN,11,31/03/2020,0.451600
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,NaN,NaN,11,31/03/2020,0.449224
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,NaN,NaN,46,06/10/2022,0.887060
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,NaN,NaN,110,15/08/2022,0.884231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,"I saw something in the physical, I saw somethi...","Meditation, Intention, CE-5 App, Telepathy",111,14/10/2023,0.005335
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,I felt an energy present and decided to take s...,"Meditation, Intention, CE-5 App",111,14/10/2023,0.005335
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,"I saw something in the physical, I saw somethi...","Telepathy, Visual contact.",182,19/06/2023,0.007973
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,"I saw something in the physical, I saw somethi...",Intention,79,11/11/2023,0.058681


In [5]:



def basic_nlp(text):
    # print("\n\n====")
    # print(text[:20])
    doc = nlp(text)
    language = doc._.language
    words = [token.text for token in doc]

    keywords = [keyword[0] for keyword in doc._.keywords]
    # print("Top Document Keywords:", doc._.keywords)


    return {"language":language['language'], 'word_count': len(words), 'keywords':", ".join(keywords)}

applied_df = df.apply(lambda row: basic_nlp(row.description), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,contact_type_raw,method_raw,sunspots,formated_date,moon_phase,language,word_count,keywords
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,NaN,NaN,11,31/03/2020,0.453401,pt,3,
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,NaN,NaN,11,31/03/2020,0.451600,en,212,star guide app
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,NaN,NaN,11,31/03/2020,0.449224,en,51,"took quick video, south west sky, noticed movi..."
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,NaN,NaN,46,06/10/2022,0.887060,en,20,"black triangles shape, triangles shape ufo, sh..."
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,NaN,NaN,110,15/08/2022,0.884231,en,23,"green light, Weird green light"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,"I saw something in the physical, I saw somethi...","Meditation, Intention, CE-5 App, Telepathy",111,14/10/2023,0.005335,en,192,"CE5 Skywatch Team, Canberra CE5 Skywatch, CE5 ..."
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,I felt an energy present and decided to take s...,"Meditation, Intention, CE-5 App",111,14/10/2023,0.005335,en,24,
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,"I saw something in the physical, I saw somethi...","Telepathy, Visual contact.",182,19/06/2023,0.007973,en,3,Strange mind invasion
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,"I saw something in the physical, I saw somethi...",Intention,79,11/11/2023,0.058681,en,154,"craft flying high, Star Families homes, relati..."


In [6]:
# from summa import summarizer
# from summa import keywords
# import nltk
# from nltk.stem import PorterStemmer

# ps = PorterStemmer()



# def summarize_text(text):
#     # print(text[:10])
#     stems = []
#     keys = keywords.keywords(text)
#     if len(keys) > 1:
#         for word in str(keywords.keywords(text)).split("\n"):
#             stems.append(ps.stem(word))
#     else:
#         stems = []
#     return {
#         'summary': summarizer.summarize(text),
#         'keywords': str(keywords.keywords(text)).replace("\n", ", "),
#         'stemmed': ", ".join(stems)
#     }
# df = df.loc[df['language'] == 'en']
# df = df.loc[df['word_count'] >5]
# applied_df = df.apply(lambda row: summarize_text(row.description), axis='columns', result_type='expand')
# df = pd.concat([df, applied_df], axis='columns')

# df



In [7]:
r = Rake()

def rake_words(description):
    r.extract_keywords_from_text(description)
    return ", ".join(r.get_ranked_phrases()[:20])
df['rake_words'] = df['description'].apply(lambda x: rake_words(str(x)))
df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,contact_type_raw,method_raw,sunspots,formated_date,moon_phase,language,word_count,keywords,rake_words
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,NaN,NaN,11,31/03/2020,0.453401,pt,3,,madar node 83
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,NaN,NaN,11,31/03/2020,0.451600,en,212,star guide app,"star guide app, 2 minutes later, would come ba..."
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,NaN,NaN,11,31/03/2020,0.449224,en,51,"took quick video, south west sky, noticed movi...","took quick video watched, noticed moving image..."
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,NaN,NaN,46,06/10/2022,0.887060,en,20,"black triangles shape, triangles shape ufo, sh...","black triangles shape ufo, ufo seemed, certain..."
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,NaN,NaN,110,15/08/2022,0.884231,en,23,"green light, Weird green light","weird green light, wanna know, video, tonight,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,"I saw something in the physical, I saw somethi...","Meditation, Intention, CE-5 App, Telepathy",111,14/10/2023,0.005335,en,192,"CE5 Skywatch Team, Canberra CE5 Skywatch, CE5 ...","ce5 contact app meditation .., canberra ce5 sk..."
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,I felt an energy present and decided to take s...,"Meditation, Intention, CE-5 App",111,14/10/2023,0.005335,en,24,,"light streaming around, us, took, photos, mome..."
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,"I saw something in the physical, I saw somethi...","Telepathy, Visual contact.",182,19/06/2023,0.007973,en,3,Strange mind invasion,strange mind invasion
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,"I saw something in the physical, I saw somethi...",Intention,79,11/11/2023,0.058681,en,154,"craft flying high, Star Families homes, relati...","time regular friends also popped, star familie..."


In [8]:
# import pytextrank
# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("textrank")


# def textrank(description):
#     r.extract_keywords_from_text(description)
#     return ", ".join(r.get_ranked_phrases()[:20])
# df['textrank'] = df['description'].apply(lambda x: textrank(str(x)))
# df

In [9]:
import re
def get_contact_types(text):

    if text == text:
        word_count = len(text.split())
        if word_count == 0:
            word_count = 1
        return {
            "saw": len(re.findall("physical", text)) / word_count > 0, 
            "mind": len(re.findall("mind", text)) / word_count > 0, 
            "heard": len(re.findall("heard", text)) / word_count > 0, 
            "embodied": len(re.findall("embodied", text)) / word_count > 0, 
        }
    else:
        return {
            "saw": False,
            "mind": False,
            "heard": False,
            "embodied": False,
        }
 
applied_df = df.apply(lambda row: get_contact_types(row.contact_type_raw), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,...,formated_date,moon_phase,language,word_count,keywords,rake_words,saw,mind,heard,embodied
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,...,31/03/2020,0.453401,pt,3,,madar node 83,False,False,False,False
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,...,31/03/2020,0.451600,en,212,star guide app,"star guide app, 2 minutes later, would come ba...",False,False,False,False
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,...,31/03/2020,0.449224,en,51,"took quick video, south west sky, noticed movi...","took quick video watched, noticed moving image...",False,False,False,False
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,...,06/10/2022,0.887060,en,20,"black triangles shape, triangles shape ufo, sh...","black triangles shape ufo, ufo seemed, certain...",False,False,False,False
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,...,15/08/2022,0.884231,en,23,"green light, Weird green light","weird green light, wanna know, video, tonight,...",False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,...,14/10/2023,0.005335,en,192,"CE5 Skywatch Team, Canberra CE5 Skywatch, CE5 ...","ce5 contact app meditation .., canberra ce5 sk...",True,True,False,False
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,...,14/10/2023,0.005335,en,24,,"light streaming around, us, took, photos, mome...",False,False,False,False
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,...,19/06/2023,0.007973,en,3,Strange mind invasion,strange mind invasion,True,True,False,False
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,...,11/11/2023,0.058681,en,154,"craft flying high, Star Families homes, relati...","time regular friends also popped, star familie...",True,True,False,True


In [10]:
import re


def has_keywords(text):
    word_count = len(text.split())
    if word_count == 0:
        word_count = 1
    return {
        "ce5": len(re.findall("ce-5|ce5|hice", text)) / word_count > 0,
        "bodymark": len(re.findall("bodymark|body mark|scar|wound", text)) / word_count> 0,
        "entity": len(re.findall("entity|alien", text)) / word_count> 0,
        "pulse": len(re.findall("pulsating|pulse", text)) / word_count> 0,
    }


applied_df = df.apply(lambda row: has_keywords(row.description), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,...,keywords,rake_words,saw,mind,heard,embodied,ce5,bodymark,entity,pulse
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,...,,madar node 83,False,False,False,False,False,False,False,False
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,...,star guide app,"star guide app, 2 minutes later, would come ba...",False,False,False,False,False,False,False,False
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,...,"took quick video, south west sky, noticed movi...","took quick video watched, noticed moving image...",False,False,False,False,False,False,False,False
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,...,"black triangles shape, triangles shape ufo, sh...","black triangles shape ufo, ufo seemed, certain...",False,False,False,False,False,False,False,False
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,...,"green light, Weird green light","weird green light, wanna know, video, tonight,...",False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,...,"CE5 Skywatch Team, Canberra CE5 Skywatch, CE5 ...","ce5 contact app meditation .., canberra ce5 sk...",True,True,False,False,False,False,False,False
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,...,,"light streaming around, us, took, photos, mome...",False,False,False,False,False,False,False,False
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,...,Strange mind invasion,strange mind invasion,True,True,False,False,False,False,False,False
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,...,"craft flying high, Star Families homes, relati...","time regular friends also popped, star familie...",True,True,False,True,False,False,False,False


In [11]:
import re
def get_method(text):
#     print("\n======" + text )
    if text == text:
        word_count = len(text.split())
        if word_count == 0:
            word_count = 1
        return {
            "app": len(re.findall("app", text))>0, 
            "meditation": len(re.findall("meditation", text))>0, 
            "intention": len(re.findall("intention", text))>0, 
            "telepathy": len(re.findall("telepathy", text))>0, 
            "astral": len(re.findall("astral", text))>0, 
            "dream": len(re.findall("dream", text))>0, 
            "plant": len(re.findall("plant", text))>0, 
            "radio": len(re.findall("radio", text))>0, 
            "channel": len(re.findall("channel", text))>0, 
        }
    else:
        return {
            "app":False, 
            "meditation":False, 
            "intention":False, 
            "telepathy":False, 
            "astral":False, 
            "dream":False, 
            "plant":False, 
            "radio":False, 
            "channel":False, 
        }
        
 

applied_df = df.apply(lambda row: get_method(row.method_raw), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,...,pulse,app,meditation,intention,telepathy,astral,dream,plant,radio,channel
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,...,False,False,False,False,False,False,False,False,False,False
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,...,False,False,False,False,False,False,False,False,False,False
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,...,False,False,False,False,False,False,False,False,False,False
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,...,False,False,False,False,False,False,False,False,False,False
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,...,False,False,False,False,False,False,False,False,False,False
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,...,False,False,False,False,False,False,False,False,False,False
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,...,False,False,False,False,False,False,False,False,False,False
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,...,False,False,False,False,False,False,False,False,False,False


In [12]:
import re
def get_shapes(text):
#     print("\n======" + text )
    word_count = len(text.split())
    if word_count == 0:
        word_count = 1
    shapes = {
        "shape_triangle": (len(re.findall("triangle|equilateral|pyramid", text)) / word_count)>0,
        "shape_orb": (len(re.findall("orb", text)) / word_count)>0,
        "shape_circle": (len(re.findall("circle|oval|sphere", text)) / word_count)>0,
        "shape_rectangle": (len(re.findall("square|rectangl", text)) / word_count)>0,
        "shape_cigar": (len(re.findall("cigar|cylinder", text)) / word_count)>0,
        "shape_fireball": (len(re.findall("fire ball|meteor|shooting star", text)) / word_count)>0,
        "shape_disk": (len(re.findall("disk|saucer", text)) / word_count)>0,
    }
    
    sorted_shapes = dict(sorted(shapes.items(), key=lambda x:x[1]))
    
    if shapes[list(sorted_shapes)[-1]] > 0:
        shapes['shape'] = list(sorted_shapes.keys())[-1].replace("shape_", "")
    else:
        shapes['shape'] = ''
    return shapes

applied_df = df.apply(lambda row: get_shapes(row.description), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,...,radio,channel,shape_triangle,shape_orb,shape_circle,shape_rectangle,shape_cigar,shape_fireball,shape_disk,shape
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,...,False,False,False,False,False,False,False,False,False,
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,...,False,False,False,False,False,False,False,False,False,
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,...,False,False,False,False,False,False,False,False,False,
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,...,False,False,True,False,False,False,False,False,False,triangle
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,...,False,False,False,False,False,False,False,False,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,...,False,False,False,False,False,False,False,False,False,
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,...,False,False,False,False,False,False,False,False,False,
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,...,False,False,False,False,False,False,False,False,False,
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,...,False,False,False,True,False,False,False,False,False,orb


In [13]:
from nrclex import NRCLex

import re
def nrc_lex(text):
    # print("\n======" + text[:10] )
    text_object = NRCLex(text)
    # print(text_object.top_emotions[0][0])
    emotions = text_object.affect_frequencies
    emotions['emotion'] = text_object.top_emotions[0][0]
    return emotions
  
 

applied_df = df.apply(lambda row: nrc_lex(row.description), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df


,updb_id,category,data_source,date,description,location,city,state,country,user_id,...,anticip,trust,surprise,positive,negative,sadness,disgust,joy,emotion,anticipation
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,fear,NaN
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,...,0.0,0.095238,0.000000,0.285714,0.190476,0.095238,0.0,0.047619,positive,0.190476
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,...,0.0,0.181818,0.000000,0.363636,0.090909,0.000000,0.0,0.090909,positive,0.181818
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,...,0.0,0.000000,0.000000,0.333333,0.333333,0.333333,0.0,0.000000,positive,NaN
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,...,0.0,0.142857,0.000000,0.142857,0.142857,0.142857,0.0,0.142857,fear,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,...,0.0,0.058824,0.117647,0.411765,0.058824,0.058824,0.0,0.058824,positive,0.235294
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,fear,NaN
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,...,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.0,0.000000,anger,NaN
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,...,0.0,0.086957,0.043478,0.478261,0.000000,0.000000,0.0,0.130435,positive,0.173913


In [14]:

# from limbic.emotion.models.tf_limbic_model import TfLimbicModel
# tf_model = TfLimbicModel()

# def get_emotions(text):
#     raw_emotions = tf_model.get_sentence_emotions(text)
#     emotions = {}
#     for emotion in raw_emotions:
#         emotions[emotion.category] = emotion.value
#     sorted_emotions = dict(sorted(emotions.items(), key=lambda x:x[1]))
#     if emotions[list(sorted_emotions)[-1]] > 0:
#         emotions['emotion'] = list(sorted_emotions.keys())[-1]
#     else:
#         emotions['emotion'] = ''
#     return emotions


# applied_df = df.apply(lambda row: get_emotions(row.description), axis='columns', result_type='expand')
# df = pd.concat([df, applied_df], axis='columns')

# df

In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()
def get_sentiment(text):
    sentiment_dict = sid_obj.polarity_scores(text)
    if sentiment_dict['compound'] >= 0.05 :
        sentiment_dict['sentiment']='positive'
    elif sentiment_dict['compound'] <= - 0.05 :
        sentiment_dict['sentiment']='negative'
    else :
        sentiment_dict['sentiment']='neutral'
    return sentiment_dict



applied_df = df.apply(lambda row: get_sentiment(row.description), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,...,sadness,disgust,joy,emotion,anticipation,neg,neu,pos,compound,sentiment
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,PUEBLO,COLORADO,US,NaN,...,0.000000,0.0,0.000000,fear,NaN,0.000,1.000,0.000,0.0000,neutral
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,CENTER CONWAY,NEW HAMPSHIRE,US,NaN,...,0.095238,0.0,0.047619,positive,0.190476,0.030,0.890,0.080,0.8883,positive
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,DRACUT,MASSACHUSETTS,US,NaN,...,0.000000,0.0,0.090909,positive,0.181818,0.000,1.000,0.000,0.0000,neutral
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,WASCO,CALIFORNIA,US,NaN,...,0.333333,0.0,0.000000,positive,NaN,0.000,0.896,0.104,0.2732,positive
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,BURY ST EDMUNDS,NaN,GB,NaN,...,0.142857,0.0,0.142857,fear,0.142857,0.078,0.922,0.000,-0.1779,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,...,0.058824,0.0,0.058824,positive,0.235294,0.000,0.930,0.070,0.8977,positive
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,...,0.000000,0.0,0.000000,fear,NaN,0.000,0.909,0.091,0.2732,positive
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,...,0.000000,0.0,0.000000,anger,NaN,0.474,0.526,0.000,-0.2023,negative
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,...,0.000000,0.0,0.130435,positive,0.173913,0.051,0.874,0.075,0.6369,positive


In [18]:
df['city'] = df['city'].apply(lambda x: str(x).title())
df['state'] = df['state'].apply(lambda x: str(x).title())
def country_codes(country):
    if country.title() == "Australia":
        return "AU"
    elif country.title() == "United States":
        return "US"
    elif country.title() == "United Kingdom":
        return "UK"
    elif country.title() == "Philippines":
        return "PH"
    elif country.title() == "Nicaragua":
        return "NI"
    elif country.title() == "Hong Kong":
        return "HK"
    elif country.title() == "Canada":
        return "CA"
    else :
        return country
df['country'] = df['country'].apply(lambda x: country_codes(str(x)))
df

,updb_id,category,data_source,date,description,location,city,state,country,user_id,...,sadness,disgust,joy,emotion,anticipation,neg,neu,pos,compound,sentiment
0,11435079.0,UAP,NUFORC,2020-03-31 23:25,MADAR Node 83,5190314.0,Pueblo,Colorado,US,NaN,...,0.000000,0.0,0.000000,fear,NaN,0.000,1.000,0.000,0.0000,neutral
1,11435082.0,UAP,NUFORC,2020-03-31 23:00,"Glowing, flickering light with reds moving in ...",11269854.0,Center Conway,New Hampshire,US,NaN,...,0.095238,0.0,0.047619,positive,0.190476,0.030,0.890,0.080,0.8883,positive
2,11435084.0,UAP,NUFORC,2020-03-31 22:27,Walking home from neighbor and noticed moving ...,5235197.0,Dracut,Massachusetts,US,NaN,...,0.000000,0.0,0.090909,positive,0.181818,0.000,1.000,0.000,0.0000,neutral
3,11449651.0,UAP,NUFORC,2022-10-06 20:09,Its was a black triangles shape ufo\n\nThe uf...,5222962.0,Wasco,California,US,NaN,...,0.333333,0.0,0.000000,positive,NaN,0.000,0.896,0.104,0.2732,positive
4,6107981.0,UAP,MUFON,2022-08-15 0:00,Weird green light\n\nJust look at my photos an...,5291710.0,Bury St Edmunds,Nan,GB,NaN,...,0.142857,0.0,0.142857,fear,0.142857,0.078,0.922,0.000,-0.1779,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44122,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,This was the first Skywatch that the Canberra ...,NaN,0,0,0,Canberra CE5 Skywatch Team,...,0.058824,0.0,0.058824,positive,0.235294,0.000,0.930,0.070,0.8977,positive
44123,NaN,HICE,ETLT_GCE5,2023-10-14 0:00,I felt an energy with us and took a few photos...,NaN,0,0,0,Sandhills CE5,...,0.000000,0.0,0.000000,fear,NaN,0.000,0.909,0.091,0.2732,positive
44124,NaN,HICE,ETLT_GCE5,2023-06-19 0:00,Strange mind invasion,NaN,0,0,0,NaN,...,0.000000,0.0,0.000000,anger,NaN,0.474,0.526,0.000,-0.2023,negative
44125,NaN,HICE,ETLT_GCE5,2023-11-11 0:00,"With Jupiter rising at sunset, & now Orion wit...",NaN,0,0,0,Friends of Ra,...,0.000000,0.0,0.130435,positive,0.173913,0.051,0.874,0.075,0.6369,positive


In [19]:
df.to_csv("analyzed_"+str(sample_size)+".csv")